In [16]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
import json
import os
import glob

In [17]:
csv_file = "data-files/Food_Inspections.csv"
food_inspections_df = pd.read_csv(csv_file)
food_inspections_df.head()

fidf=food_inspections_df.rename(columns={'AKA Name':'name','Address':'address1'})

In [18]:
facility_type_df=food_inspections_df["Facility Type"].value_counts()
facility_type_df

Restaurant                      134372
Grocery Store                    25400
School                           12514
Children's Services Facility      3263
Bakery                            2938
                                 ...  
SPA                                  1
GROCERY & RESTAURANT                 1
PEDDLER                              1
MOBILE DESSERT VENDOR                1
FARMER'S MARKET                      1
Name: Facility Type, Length: 495, dtype: int64

In [19]:

pd.set_option('display.max_columns', None)

temp = pd.DataFrame()

path_to_json = 'data-files/json/' 

json_pattern = os.path.join(path_to_json,'*.json')
file_list = glob.glob(json_pattern)


for file in file_list:
    data = json.load(open(file))
    pd.read_json(file, lines=True)
    temp=temp.append(data["businesses"], ignore_index = True)

In [20]:
temp.head(10)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,Girl & the Goat,https://s3-media1.fl.yelpcdn.com/bphoto/ya6gjD...,False,https://www.yelp.com/biz/girl-and-the-goat-chi...,8544,"[{'alias': 'newamerican', 'title': 'American (...",4.5,"{'latitude': 41.884176, 'longitude': -87.64794...",[delivery],$$$,"{'address1': '809 W Randolph St', 'address2': ...",+13124926262,(312) 492-6262,3396.525043
1,cKZNbMvoqJaUe7n6lf6i7w,wildberry-pancakes-and-cafe-chicago-2,Wildberry Pancakes and Cafe,https://s3-media3.fl.yelpcdn.com/bphoto/or0SLp...,False,https://www.yelp.com/biz/wildberry-pancakes-an...,7276,"[{'alias': 'pancakes', 'title': 'Pancakes'}, {...",4.5,"{'latitude': 41.884668, 'longitude': -87.62288}","[pickup, delivery]",$$,"{'address1': '130 E Randolph St', 'address2': ...",+13129389777,(312) 938-9777,5082.221755
2,xoi7Cw7FoknAx5p880RtWQ,au-cheval-chicago,Au Cheval,https://s3-media4.fl.yelpcdn.com/bphoto/a0yhvS...,False,https://www.yelp.com/biz/au-cheval-chicago?adj...,7015,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"{'latitude': 41.88466, 'longitude': -87.647668}",[delivery],$$,"{'address1': '800 W Randolph St', 'address2': ...",+13129294580,(312) 929-4580,3376.598427
3,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,The Purple Pig,https://s3-media2.fl.yelpcdn.com/bphoto/Qf9YAh...,False,https://www.yelp.com/biz/the-purple-pig-chicag...,7130,"[{'alias': 'tapasmallplates', 'title': 'Tapas/...",4.0,"{'latitude': 41.890694, 'longitude': -87.624782}","[pickup, delivery]",$$,"{'address1': '444 N Michigan Ave', 'address2':...",+13124641744,(312) 464-1744,4670.876907
4,8vFJH_paXsMocmEO_KAa3w,lou-malnatis-pizzeria-chicago,Lou Malnati's Pizzeria,https://s3-media1.fl.yelpcdn.com/bphoto/cv0OrG...,False,https://www.yelp.com/biz/lou-malnatis-pizzeria...,6055,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 41.890357, 'longitude': -87.633704}","[pickup, delivery]",$$,"{'address1': '439 N Wells St', 'address2': '',...",+13128289800,(312) 828-9800,4000.999053
5,LPwAwxEjetjdNh7Uadro3g,smoque-bbq-chicago,Smoque BBQ,https://s3-media4.fl.yelpcdn.com/bphoto/T-oNCs...,False,https://www.yelp.com/biz/smoque-bbq-chicago?ad...,4077,"[{'alias': 'bbq', 'title': 'Barbeque'}]",4.5,"{'latitude': 41.95021, 'longitude': -87.72795}",[],$$,"{'address1': '3800 N Pulaski Rd', 'address2': ...",+17735457427,(773) 545-7427,6499.895336
6,y8jVBaHh8ntYkVqDXpsi1Q,cafe-ba-ba-reeba-chicago-3,Cafe Ba-Ba-Reeba!,https://s3-media3.fl.yelpcdn.com/bphoto/31Kpq5...,False,https://www.yelp.com/biz/cafe-ba-ba-reeba-chic...,4038,"[{'alias': 'spanish', 'title': 'Spanish'}, {'a...",4.0,"{'latitude': 41.918984, 'longitude': -87.648687}","[pickup, delivery]",$$,"{'address1': '2024 N Halsted St', 'address2': ...",+17739355000,(773) 935-5000,2855.274609
7,apyIymDFbUK7u-Y3V5kmkA,bavettes-bar-and-boeuf-chicago,Bavette's Bar & Boeuf,https://s3-media3.fl.yelpcdn.com/bphoto/e-0zil...,False,https://www.yelp.com/biz/bavettes-bar-and-boeu...,2334,"[{'alias': 'steak', 'title': 'Steakhouses'}, {...",4.5,"{'latitude': 41.8892796234574, 'longitude': -8...",[delivery],$$$,"{'address1': '218 W Kinzie St', 'address2': ''...",+13126248154,(312) 624-8154,3968.647508
8,pbEiXam9YJL3neCYHGwLUA,alinea-chicago,Alinea,https://s3-media2.fl.yelpcdn.com/bphoto/5v9Cwg...,False,https://www.yelp.com/biz/alinea-chicago?adjust...,2171,"[{'alias': 'newamerican', 'title': 'American (...",4.5,"{'latitude': 41.9134194160823, 'longitude': -8...",[],$$$$,"{'address1': '1723 N Halsted St', 'address2': ...",+13128670110,(312) 867-0110,2610.289489
9,-sdMTRmgExufOk6STZFi9w,little-goat-diner-chicago-4,Little Goat Diner,https://s3-media3.fl.yelpcdn.com/bphoto/FEdL5J...,False,https://www.yelp.com/biz/little-goat-diner-chi...,4518,"[{'alias': 'diners', 'title': 'Diners'}]",4.0,"{'latitude': 41.8846460590586, 'longitude': -8...",[delivery],$$,"{'address1': '820 W Randolph S

In [6]:
df2 = temp[['name', 'review_count','rating','price','coordinates','location']].copy()
df2

,name,review_count,rating,price,coordinates,location
0,Girl & the Goat,8544,4.5,$$$,"{'latitude': 41.884176, 'longitude': -87.64794...","{'address1': '809 W Randolph St', 'address2': ..."
1,Wildberry Pancakes and Cafe,7276,4.5,$$,"{'latitude': 41.884668, 'longitude': -87.62288}","{'address1': '130 E Randolph St', 'address2': ..."
2,Au Cheval,7015,4.5,$$,"{'latitude': 41.88466, 'longitude': -87.647668}","{'address1': '800 W Randolph St', 'address2': ..."
3,The Purple Pig,7130,4.0,$$,"{'latitude': 41.890694, 'longitude': -87.624782}","{'address1': '444 N Michigan Ave', 'address2':..."
4,Lou Malnati's Pizzeria,6055,4.0,$$,"{'latitude': 41.890357, 'longitude': -87.633704}","{'address1': '439 N Wells St', 'address2': '',..."
...,...,...,...,...,...,...
995,Wakamono,508,3.5,$$,"{'latitude': 41.9423546, 'longitude': -87.6442...","{'address1': '3313 N Broadway Ave', 'address2'..."
996,Nando's Peri-Peri,227,4.0,$$,"{'latitude': 41.893239, 'longitude': -87.62162...","{'address1': '227 E Ontario St', 'address2': '..."
997,Han 202,276,4.0,$$,"{'latitude': 41.837927, 'longitude': -87.641755}","{'address1': '605 W 31st St', 'address2': '', ..."
998,Bijan's Bistro,560,3.5,$$,"{'latitude': 41.8940736918469, 'longitude': -8...","{'address1': '663 N State St', 'address2': '',..."


In [7]:
latitude=[]
longitude=[]
address1=[]
address2=[]
address3=[]
city=[]
zip_code=[]
country=[]
state=[]

for k,v in df2.iterrows():
    latitude.append(v.coordinates['latitude'])
    longitude.append(v.coordinates['longitude'])
    address1.append(v.location['address1'])
    address2.append(v.location['address2'])
    address3.append(v.location['address3'])
    city.append(v.location['city'])
    zip_code.append(v.location['zip_code'])
    country.append(v.location['country'])
    state.append(v.location['state'])
        


In [8]:
df2['latitude'] = latitude
df2['longitude'] = longitude
df2['address1'] = address1
df2['address2'] = address2
df2['address3'] = address3
df2['city'] = city
df2['zip_code'] = zip_code
df2['country'] = country
df2['state'] = state

In [9]:
df2.head()
df2.columns

Index(['name', 'review_count', 'rating', 'price', 'coordinates', 'location',
       'latitude', 'longitude', 'address1', 'address2', 'address3', 'city',
       'zip_code', 'country', 'state'],
      dtype='object')

In [10]:
df3 = df2[['name', 'review_count','rating','price','latitude', 'longitude', 'address1', 'address2', 'address3', 'city',
       'zip_code', 'country', 'state']].copy()
(df3['name'] == "Girl & the Goat").value_counts()

False    999
True       1
Name: name, dtype: int64

In [11]:
df3

,name,review_count,rating,price,latitude,longitude,address1,address2,address3,city,zip_code,country,state
0,Girl & the Goat,8544,4.5,$$$,41.884176,-87.647944,809 W Randolph St,,,Chicago,60607,US,IL
1,Wildberry Pancakes and Cafe,7276,4.5,$$,41.884668,-87.622880,130 E Randolph St,,,Chicago,60601,US,IL
2,Au Cheval,7015,4.5,$$,41.884660,-87.647668,800 W Randolph St,,,Chicago,60607,US,IL
3,The Purple Pig,7130,4.0,$$,41.890694,-87.624782,444 N Michigan Ave,None,,Chicago,60611,US,IL
4,Lou Malnati's Pizzeria,6055,4.0,$$,41.890357,-87.633704,439 N Wells St,,,Chicago,60654,US,IL
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Wakamono,508,3.5,$$,41.942355,-87.644283,3313 N Broadway Ave,,,Chicago,60657,US,IL
996,Nando's Peri-Peri,227,4.0,$$,41.893239,-87.621628,227 E Ontario St,,None,Chicago,60611,US,IL
997,Han 202,276,4.0,$$,41.837927,-87.641755,605 W 31st St,,,Chicago,60616,US,IL
998,Bijan's Bistro,560,3.5,$$,41.894074,-87.627912,663 N State St,,,Chicago,60654,US,IL


In [12]:
df3["address2"].unique()
df3["address3"].unique()

array(['', None, 'Peninsula Chicago Hotel', 'Park Hyatt Chicago',
       'Museum Campus', 'Chicago French Market', 'Lobby Level',
       'Water Tower Pl', 'Macy’s Seven on State'], dtype=object)

In [23]:
dfand=df3[df3['name'].str.contains("&")]
dfdash=df3[df3['name'].str.contains("-")]
dfand

,name,review_count,rating,price,latitude,longitude,address1,address2,address3,city,zip_code,country,state
0,Girl & the Goat,8544,4.5,$$$,41.884176,-87.647944,809 W Randolph St,,,Chicago,60607,US,IL
7,Bavette's Bar & Boeuf,2334,4.5,$$$,41.889280,-87.634823,218 W Kinzie St,,,Chicago,60654,US,IL
15,"Joe's Seafood, Prime Steak & Stone Crab",2431,4.5,$$$,41.891938,-87.625690,60 E Grand Ave,,None,Chicago,60611,US,IL
27,Batter & Berries,2631,4.0,$$,41.931561,-87.657791,2748 N Lincoln Ave,,,Chicago,60614,US,IL
32,Longman & Eagle,2394,4.0,$$,41.930157,-87.707222,2657 N Kedzie Ave,,,Chicago,60647,US,IL
...,...,...,...,...,...,...,...,...,...,...,...,...,...
934,Baptiste & Bottle,339,4.0,$$,41.893975,-87.625236,101 E Erie St,Fl 20,None,Chicago,60611,US,IL
952,G & O,201,4.0,$$,41.890556,-87.659189,459 W Ogden Ave,,,Chicago,60642,US,IL
970,Stan's Donuts & Coffee,296,4.0,$,41.885377,-87.624226,181 N Michigan Ave,,None,Chicago,60601,US,IL
972,Chicken Works & Salad Company,232,4.0,$,41.953891,-87.719712,3658 W Irving Park Rd,,,Chicago,60618,US,IL


In [32]:
merged_df = df3.join(fidf, how = 'inner', on = ['address1'])
merged_df

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [31]:
merged_df['name_x'].unique()

array(['Edgewater Tacos'], dtype=object)